# Input data

In [12]:
import gurobipy as gb
import numpy as np
import pandas as pd

gen_costs = pd.read_excel('gen_costs.xlsx')
gen_data = pd.read_excel('gen_data.xlsx')
line_data = pd.read_excel('line_data.xlsx')
system_demand = pd.read_excel('system_demand.xlsx')
load_distribution = pd.read_excel('load_distribution.xlsx')

wind_data = []
for i in range(6):
    wind_data.append(pd.read_csv('wind 1.out'))


n_bus = 24

#per-unit bases
S_base_3ph = 100 #MW

bus_data = pd.DataFrame(index = np.arange(n_bus), data = np.zeros(n_bus), columns = ['Load'])
bus_data['Wind'] = 0

line_data['Susceptance pu'] = 1 / line_data['Reactance pu']
line_data['Capacity pu'] = line_data['Capacity MVA'] / S_base_3ph
branch_matrix = np.zeros((n_bus, n_bus)) #susceptance

#Filling out susceptance matrix

for n in range(1, n_bus + 1):
    branch_matrix[n - 1][n - 1] = line_data.loc[(line_data['From'] == n) | (line_data['To'] == n), 'Susceptance pu'].sum()
    for k in range(n, n_bus + 1): #start from n
        if k != n:
            branch_matrix[n - 1][k - 1] = -1 * (line_data.loc[(line_data['From'] == n) & (line_data['To'] == k), 'Susceptance pu'].sum())
            branch_matrix[k - 1][n - 1] = branch_matrix[n - 1][k - 1]

# Task 1 (single time step)

In [2]:
#Choosing to run for the first hour
load = np.zeros(n_bus)
demand = system_demand['System Demand'][0]

for n in load_distribution['Node'].unique():
    load[n] = load_distribution.loc[load_distribution['Node'] == n, '% of system load'] / 100 * demand

load

C:\Users\Markus\AppData\Local\Temp\ipykernel_10984\3973589358.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  load[n] = load_distribution.loc[load_distribution['Node'] == n, '% of system load'] / 100 * demand


array([  0.      ,  67.48173 ,  60.37839 , 111.877605,  46.17171 ,
        44.395875,  85.24008 ,  78.13674 , 106.5501  , 108.325935,
       120.75678 ,   0.      ,   0.      , 165.152655, 120.75678 ,
       197.117685,  62.154225,   0.      , 207.772695, 113.65344 ,
        79.912575,   0.      ,   0.      ,   0.      ])

Setting up a dictionary which maps bus indices to generator indices for easier looping - note that it accounts for zero-indexing, so the generator indices go from 0 to 11

In [3]:
gens_map = {}

for n in range(1, n_bus + 1):
    gens_map[n - 1] = (gen_data['Unit #'][gen_data['Node'] == n] - 1).tolist()

In [4]:
gens_map.get(14) #example: get the generator indices at bus 14 (15 in the assignment formulation)

[4, 5]

### Setup model and solve (currently returning infeasible)

In [23]:
direction = gb.GRB.MINIMIZE #Min / Max

# Create a Gurobi model       
m = gb.Model()

# Add variables
p_G = m.addVars(len(gen_data.index), lb=0, ub=gb.GRB.INFINITY, name="P_G") #Note: This is in per unit
theta = m.addVars(n_bus, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="theta")

# Set objective function
obj = gb.quicksum(gen_costs['C ($/MWh)'][k] * p_G[k] * S_base_3ph for k in range(len(p_G)))         
m.setObjective(obj, direction) #Minimization / Maximization


#Setting up balance equation for each node using the gens_map dictionary
for i in range(n_bus):
    m.addConstr(gb.quicksum(p_G[g] for g in gens_map.get(i)) - load[i] == theta[i] * branch_matrix[i,i] + gb.quicksum(theta[k] * branch_matrix[i,k] for k in range(n_bus) if k != i))

#Managing line capacities - remembering that the "to" and "from" are not zero-indexed in the data
for i in range(n_bus):
    for k in range(i, n_bus): #Avoid duplicates by starting the indexing of k at i
        if (i != k) and (branch_matrix[i,k] != 0):
            m.addConstr((theta[i] - theta[k]) * branch_matrix[i,k] <= (line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()))
            m.addConstr((theta[i] - theta[k]) * branch_matrix[i,k] >= -1 * (line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()))

m.addConstrs(p_G[i] <= (gen_data['P max MW'].iloc[i] / S_base_3ph) for i in range(len(gen_data)))
m.addConstrs(p_G[i] >= (gen_data['P min MW'].iloc[i] / S_base_3ph) for i in range(len(gen_data)))
m.addConstr(theta[0] == 0)

m.update()
m.display()

m.optimize()

Minimize
1332.0 P_G[0] + 1332.0 P_G[1] + 2070.0 P_G[2] + 2093.0 P_G[3] + 2611.0 P_G[4]
+ 1052.0 P_G[5] + 1052.0 P_G[6] + 602.0 P_G[7] + 547.0 P_G[8] + 1052.0 P_G[10]
+ 1089.0 P_G[11]
Subject To
R0: P_G[0] + -83.95703541830918 theta[0] + 68.4931506849315 theta[1] +
 4.438526409232135 theta[2] + 11.025358324145534 theta[4] = 0
R1: P_G[1] + 68.4931506849315 theta[0] + -80.74583073385588 theta[1] +
 7.374631268436579 theta[3] + 4.878048780487805 theta[5] = 67.4817
R2: 4.438526409232135 theta[0] + -24.211108927684045 theta[2] + 7.867820613690008
 theta[8] + 11.904761904761903 theta[23] = 60.3784
R3: 7.374631268436579 theta[1] + -16.383640277445586 theta[3] + 9.00900900900901
 theta[8] = 111.878
R4: 11.025358324145534 theta[0] + -21.66365619648596 theta[4] + 10.638297872340425
 theta[9] = 46.1717
R5: 4.878048780487805 theta[1] + -20.454372768026747 theta[5] + 15.576323987538942
 theta[9] = 44.3959
  R6: P_G[2] + -15.337423312883438 theta[6] + 15.337423312883438 theta[7] = 85.2401
R7: 15.3374

### Run model

In [22]:
#Print solutions
if m.status == gb.GRB.OPTIMAL:
    constraints = m.getConstrs()
    # The constraint dual value / sensitivity in the current solution (also known as the shadow price)... https://www.gurobi.com/documentation/9.5/refman/pi.html
    optimized_sens = [constraints[k].Pi for k in range(len(constraints))] 
    print('-----------------------------------------------')
    print("Optimal objective value: %.2f DKK" % m.objVal)

    for i in range(n_bus):
        print(p_G[i].VarName + ": %.2f pu" % p_G[i].x)
        print(theta[i].VarName + ": %.2f rad" % theta[i].x)

    for k in range(len(constraints)):
        print('Dual value {0}: '.format(k+1), optimized_sens[k])

    print("\nFlows:")
    for i in range(n_bus):
        for k in range(i, n_bus):
            if (i != k) and (branch_matrix[i,k] != 0):
                print('From %d to %d: %.2f pu' % (i + 1, k + 1, (theta[i].x - theta[k].x) * branch_matrix[i,k]))
    for i in range(n_bus):
        net_flow = 0
        for k in range(n_bus):
            if (i != k) and (branch_matrix[i,k] != 0):
                net_flow += (theta[i].x - theta[k].x) * branch_matrix[i,k]
        print('Bus %d net injection: %.2f pu' % (i + 1, net_flow))

    print("\nTotal load: %.2f MWh" % (sum(load[n] for n in range(n_bus)) * S_base_3ph))
    print("Total generation: %.2f MWh" % (sum(p_G[n].x for n in range(n_bus)) * S_base_3ph))

else:
    print("Optimization was not successful.")     
  
        
# Dispose of the Gurobi model to release resources
m.dispose()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)



CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 117 rows, 36 columns and 265 nonzeros
Model fingerprint: 0x6acdbd59
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+02]
Presolve removed 31 rows and 2 columns
Presolve time: 0.02s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Infeasible model
Optimization was not successful.
